# Funciones generadoras
Por regla general, cuando queremos crear una lista de algún tipo, lo que hacemos es crear la lista vacía, y luego con un bucle varios elementos e ir añadiendolos a la lista si cumplen una condición:

In [8]:
[numero for numero in [0,1,2,3,4,5,6,7,8,9,10] if numero % 2 == 0 ]    

[0, 2, 4, 6, 8, 10]

También vimos cómo era posible utilizar la función **range()** para generar dinámicamente la lista en la memoria, es decir, no teníamos que crearla en el propio código, sino que se interpretaba sobre la marcha:

In [9]:
[numero for numero in range(0,11) if numero % 2 == 0 ]

[0, 2, 4, 6, 8, 10]

La verdad es que **range()** es una especie de función generadora. Por regla general las funciones devolvuelven un valor con **return**, pero la preculiaridad de los generadores es que van *cediendo* valores sobre la marcha, en tiempo de ejecución.

La función generadora **range(0,11)**, empieza cediendo el **0**, luego se procesa el for comprobando si es par y lo añade a la lista, en la siguiente iteración se cede el **1**, se procesa el for se comprueba si es par, en la siguiente se cede el **2**, etc. 

Con esto se logra ocupar el mínimo de espacio en la memoria y podemos generar listas de millones de elementos sin necesidad de almacenarlos previamente. 

Veamos a ver cómo crear una función generadora de pares:

In [30]:
def pares(n):
    for numero in range(n+1):
        if numero % 2 == 0:
            yield numero
        
pares(10)

<generator object pares at 0x000002945F38BFC0>

Como vemos, en lugar de utilizar el **return**, la función generadora utiliza el **yield**, que significa ceder. Tomando un número busca todos los pares desde 0 hasta el número+1 sirviéndonos de un range(). 

Sin embargo, fijaros que al imprimir el resultado, éste nos devuelve un objeto de tipo generador.

De la misma forma que recorremos un **range()** podemos utilizar el bucle for para recorrer todos los elementos que devuelve el generador:

In [21]:
for numero in pares(10):
    print(numero)

0
2
4
6
8
10


Utilizando comprensión de listas también podemos crear una lista al vuelo:

In [22]:
[numero for numero in pares(10)]

[0, 2, 4, 6, 8, 10]

Sin embargo el gran potencial de los generadores no es simplemente crear listas, de hecho como ya hemos visto, el propio resultado no es una lista en sí mismo, sino una __**secuencia iterable**__ con un montón de características únicas.

## Iteradores
Por tanto las funciones generadoras devuelven un objeto que suporta un protocolo de iteración. ¿Qué nos permite hacer? Pues evidentemente controlar el proceso de generación. Teniendo en cuenta que cada vez que la función generadora cede un elemento, queda suspendida y se retoma el control hasta que se le pide generar el siguiente valor. 

Así que vamos a tomar nuestro ejemplo de pares desde otra perspectiva, como si fuera un iterador manual, así veremos exactamente a lo que me refiero:

In [31]:
pares = pares(3)

Bien, ahora tenemos un iterador de pares con todos los números pares entre el 0 y el 3. Vamos a conseguir el primer número par:

In [32]:
next(pares)

0

Como vemos la función integrada **next()** nos permite acceder al siguiente elemento de la secuencia. Pero no sólo eso, si volvemos a ejecutarla...

In [33]:
next(pares)

2

Ahora devuelve el segundo! ¿No os recuerdo esto al puntero de los ficheros? Cuando leíamos una línea, el puntero pasaba a la siguiente y así sucesivamente. Pues aquí igual. 

¿Y qué pasaría si intentamos acceder al siguiente, aún sabiendo que entre el 0 y el 3 sólo tenemos los pares 0 y 2?

In [34]:
next(pares)

StopIteration: 

Pues que nos da un error porque se ha acabado la secuencia, así que tomad nota y capturad la excepción si váis a utilizarlas sin saber exactamente cuantos elementos os devolverá el generador.

Así que la pregunta que nos queda es ¿sólo es posible iterar secuencias generadas al vuelo? Vamos a probar con una lista:

In [38]:
lista = [1,2,3,4,5]
next(lista)

TypeError: 'list' object is not an iterator

¿Quizá con una cadena?

In [39]:
cadena = "Hola"
next(cadena)

TypeError: 'str' object is not an iterator

Pues no, no podemos iterar ninguna colección como si fuera una secuencia. Sin embargo, hay una función muy interesante que nos permite convertir las cadenas y algunas colecciones a *objetos iteradores*, la función **iter()**:

In [9]:
lista = [1,2,3,4,5]
lista_iterador = iter(lista)

print( next(lista_iterador) )
print( next(lista_iterador) )
print( next(lista_iterador) )
print( next(lista_iterador) )
print( next(lista_iterador) )

1
2
3
4
5


Un objeto se dice que es **iterable**, si podemos obtener un **objeto iterador** de él y, por tanto, puede ser **iterado**. La mayoría de los contenedores integrados en Python son iterables, como por ejemplo: **list**, **tuple**, **string**, etc...

Técnicamente hablando, un objeto iterable debe implementar dos métodos especiales **\__iter\__()** y **\__next\__()**, a lo que se llama **protocolo iterador**.

La llamada a la función **iter(objeto)** se corresponde con **objeto.\__iter\__()**, y devuelve un iterador, a partir del objeto iterable. De forma similar, la llamada a la función **next(objeto)**, es sinónimo a llamar al método implementado del objeto iterable **objeto.\__next\__()**. 

In [10]:
cadena = "Hola"
#cadena_iterable = iter(cadena)
cadena_iterador = cadena.__iter__()

print( next(cadena_iterador) )
print( next(cadena_iterador) )
print( cadena_iterador.__next__() )
print( cadena_iterador.__next__() )

H
o
l
a


El bucle **for** puede iterar sobre cualquier iterable (**list**, **tuple**, **string**,...). Echemos un vistazo más de cerca a cómo se implementa el bucle **for** en Python.

In [ ]:
iterable = [1, 2, 3]

for element in iterable:
    # Especificamos la lógica a realizar
    pass

En realidad, el bucle **for** de manera interna se implementa como:

In [13]:
iterable = [1,2,3,4,5]
objeto_iterador = iter(iterable) # Convertimos la lista a una secuencia iterable
#print(objeto_iterador)

while True:
    try:
        elemento = next(objeto_iterador)
        print(elemento)
    except StopIteration:
        break

1
2
3
4
5


El bucle **for**, de manera interna, crea un **objeto iterador** (objeto_iterador), llamando a la función **iter()** sobre el **iterable**. Dentro del bucle, la llamada a **next()** proporcionará el siguiente elemento ejecutándose el cuerpo del bucle **for** para ese valor. Al terminar de recorrer todos los elementos, se generará internamente una excepción del tipo **StopIteration** y el bucle finalizará.

## Construyendo tu propio iterador en Python

Construir un iterador desde cero es fácil en Python. Solo tenemos que implementar los métodos **\__iter\__()** y **\__next\__()**.

El método **\__iter\__()** devuelve el objeto iterador en sí mismo. Si es necesario, se puede realizar alguna inicialización. El método **\__next\__()** debe devolver el siguiente elemento de la secuencia. Al llegar al final, y en llamadas posteriores, debería generarse un **StopIteration**.

Aquí, mostramos un ejemplo que nos dará la próxima potencia de 2 en cada iteración. El exponente de potencia comienza desde cero hasta un número establecido por el usuario.

In [17]:
class Potencia_Dos:
    """Clase que implemente un iterador de la potencia del 2"""

    def __init__(self, max = 0):
        self.max = max

    def __iter__(self):
        self.n = 0
        return self

    def __next__(self):
        if self.n <= self.max:
            resultado = 2 ** self.n
            self.n += 1
            return resultado
        else:
            raise StopIteration

In [18]:
for i in Potencia_Dos(5):
    print(i)

1
2
4
8
16
32


O la función de los **Pares** que se definió usando los generadores.

In [51]:
class Pares:
    """Clase que implemente un iterador de números pares"""

    def __init__(self, max = 0):
        self.max = max

    def __iter__(self):
        self.n = 0
        return self

    def __next__(self):
        if self.n <= self.max:
            if self.n % 2 == 0:
                resultado = self.n
                self.n += 1
                return resultado
            else:
                self.n += 1
                return self.__next__()
        else:
            raise StopIteration
            
    # De forma más sencilla
    # def __next__(self):
    #    if self.n <= self.max:
    #        resultado = self.n
    #        self.n +=2
    #        return resultado
    #    else:
    #        raise StopIteration

In [52]:
for i in Pares(10):
    print(i)

0
2
4
6
8
10


No obstante, la manera más aconsejable y sencilla de crear **Iteradores** es mediante el uso de *Generadores*. Y a modo de recordatorio se definían:

In [21]:
def pares(n):
    for numero in range(n+1):
        if numero % 2 == 0:
            yield numero
        

In [22]:
for numero in pares(10):
    print(numero)

0
2
4
6
8
10


Muy bien, ahora ya sabemos qué son las funciones generadores, cómo utilizarlas, y también como como convertir algunos objetos a iteradores. Os sugiero probar por vuestra cuenta más colecciones a ver si encontráis alguna más que se pueda iterar.